 # **Some Regards on using Physics-Informed Neural Networks for Solving Two-Dimensional Elasticity Problems**
 
 **Authors:** Estevão Fuzaro de Almeida & Samuel da Silva
 
 **Last modified:** January 28, 2025
 
 ---

## **Case 1: plate under a vertical triangular load**

Consider a triangular plate with a linearly increasing load as shown in the figure below, with the stress $q_0 (\textrm{N/m}^2)$ at $x=L$.

<center><img src="imgs/placa1_bg.jpg"  width="400" height="450"></center>

The idea is to use the elastic equilibrium equations and find the stresses of the plate using a physics-informed neural network. These conservation equations are given by:
$$
\begin{matrix}
\frac{\partial \sigma_x}{\partial x} + \frac{\partial \tau_{xy}}{\partial y} = 0 \\
\frac{\partial \tau_{xy}}{\partial x} + \frac{\partial \sigma_y}{\partial y} = 0 \\
\frac{\partial^2 \sigma_x}{\partial x^2} + \frac{\partial^2 \sigma_y}{\partial x^2} + \frac{\partial^2 \sigma_x}{\partial y^2} + \frac{\partial^2 \sigma_y}{\partial y^2} = 0
\end{matrix}
$$
which can be simplified into two main equations:
$$
\begin{matrix}
\nabla\cdot\sigma = 0 \\
\nabla^2\left(\sigma_x + \sigma_y\right) = 0
\end{matrix}
$$

The boundary conditions can be easily obtained by evaluating faces $AB$ and $AC$ (face $BC$ is only used to find stresses at the clamping, so it is not as important).
* **Face $AB$**:
  * $y=0$
  * $∀x$
  * $0 \leq x \leq L$

$$\hat{n}_{AB} = -\hat{j};~~~~\vec{t}_{AB} = \left[\sigma\right]\hat{n}_{AB} ⇒$$

$$\vec{t}_{AB} = \begin{bmatrix} \sigma_x & \tau_{xy} \\ \tau_{xy} & \sigma_y\end{bmatrix}\begin{Bmatrix}0 \\ -1\end{Bmatrix} = \begin{Bmatrix}0 \\ q(x)\end{Bmatrix},~\textrm{where}~q(x) = -\frac{q_0}{L}x$$

$$\therefore \boxed{\tau_{xy}(x,0) = 0}~~\textrm{and}~~\boxed{\sigma_y(x,0)=\frac{q_0}{L}x}$$

* **Face $AC$**:
  * $y=x$
  * $0 \leq x \leq L$

$$\hat{n}_{AC} = -\frac{1}{\sqrt{2}}\hat{i} + \frac{1}{\sqrt{2}}\hat{j};~~~~\vec{t}_{AC} = \left[\sigma\right]\hat{n}_{AC} ⇒$$

$$\vec{t}_{AC} = \begin{bmatrix} \sigma_x & \tau_{xy} \\ \tau_{xy} & \sigma_y\end{bmatrix}\begin{Bmatrix}-\frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}}\end{Bmatrix} = \begin{Bmatrix}0 \\ 0\end{Bmatrix}$$

$$\therefore \boxed{\sigma_x(x,x) = \tau_{xy}(x,x)}~~\textrm{and}~~\boxed{\sigma_y(x,x) = \tau_{xy}(x,x)}$$

### **1. Initialization**

In [1]:
import numpy as np
import sciann as sn
import os
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from matplotlib import rc
from tensorflow import keras as ks
import subprocess

# Configure the seed for uniform network initialization
seed = 1998
np.random.seed(seed)
sn.tf.random.set_seed(seed)

# Check GPU availability and use it if available
gpu_available = sn.tf.config.list_physical_devices('GPU')
if gpu_available:
    print("TensorFlow v.", sn.tf.__version__)
    print("Number of Available GPUs:", len(sn.tf.config.experimental.list_physical_devices('GPU')))
    command = "nvidia-smi --query-gpu=name,memory.total --format=csv,noheader,nounits"
    output = subprocess.check_output(command, shell=True, encoding="utf-8")
    lines = output.strip().split("\n")
    for idx, line in enumerate(lines):
        name, memory = line.strip().split(",")
        print(f"GPU {idx} - Name: {name}, Total Memory: {memory} MB")
    sn.tf.config.experimental.set_memory_growth(gpu_available[0], True)
    print("GPU Loaded:",name)

# Configuring fonts for the plots based on LaTeX
rc('font', **{'family': 'serif', 'serif': ['Computer Modern'],'size': '18'})
rc('text', usetex=True)

---------------------- SCIANN 0.7.0.1 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.10.0 
Python Version: 3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)] 

TensorFlow v. 2.10.0
Number of Available GPUs: 1
GPU 0 - Name: NVIDIA GeForce RTX 3050 Laptop GPU, Total Memory:  4096 MB
GPU Loaded: NVIDIA GeForce RTX 3050 Laptop GPU
